In [ ]:
import csv

import numpy as np
import tensorflow as tf

RANDOM_SEED = 42

# Path

In [ ]:
model_save_path = 'model/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier.tflite'

images_dir = 'content/drive/MyDrive/augmented/train/images/'
test_dir = 'content/drive/MyDrive/augmented/test/images/'

# Number of classes

In [ ]:
NUM_CLASSES = 28

# Training Data 

In [ ]:
# X_train = np.loadtxt(train_dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
# X_test = np.loadtxt(test_dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

import os
import cv2
import numpy as np

num_images = len([f for f in os.listdir(images_dir) if f[0].isdigit()])

print(num_images)

# image_shape = (num_images, 128, 128, 3)

# X_train = np.empty(image_shape, dtype=np.uint8)
# y_train = np.empty(num_images, dtype=np.int8)

X_train = []
y_train = []

index = 0

# Loop through each image in the dataset directory
for image_name in os.listdir(images_dir):
    if image_name[0].isdigit():  # Check if the first character is a digit
        image_path = os.path.join(images_dir, image_name)

        # Extract class label
        class_label = int(image_name[0])

        # read image
        image = cv2.imread(image_path)

        resized_image = cv2.resize(image, (128, 128))

        X_train.append(resized_image)
        y_train.append(class_label)

        print(index)
        index+=1


X_train = np.array(X_train)
y_train = np.array(y_train)




# Testing Data

In [ ]:
# y_train = np.loadtxt(train_dataset, delimiter=',', dtype='int32', usecols=(0))
# y_test = np.loadtxt(test_dataset, delimiter=',', dtype='int32', usecols=(0))

import os
import cv2
import numpy as np

num_images = len([f for f in os.listdir(test_dir) if f[0].isdigit()])

print(num_images)

# image_shape = (num_images, 128, 128, 3)

# X_train = np.empty(image_shape, dtype=np.uint8)
# y_train = np.empty(num_images, dtype=np.int8)

X_test = []
y_test = []

index = 0

# Loop through each image in the dataset directory
for image_name in os.listdir(test_dir):
    if image_name[0].isdigit():  # Check if the first character is a digit
        image_path = os.path.join(test_dir, image_name)

        # Extract class label
        class_label = int(image_name[0])

        # read image
        image = cv2.imread(image_path)

        resized_image = cv2.resize(image, (128, 128))

        X_test.append(resized_image)
        y_test.append(class_label)

        print(index)
        index+=1


X_test = np.array(X_test)
y_test = np.array(y_test)

print(len(X_test), len(y_test))


# Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
# Define the input shape
input_shape = (128, 128, 3)
y_train_onehot = to_categorical(y_train, num_classes=28)
# Create a Sequential model or a Functional API model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Add fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Modify the final output layer for 28 classes
model.add(Dense(28, activation='softmax'))



# Compile model

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  # Assuming you have one-hot encoded labels
              metrics=['accuracy'])

model.compile(optimizer=Adam(learning_rate=0.0001), loss=categorical_crossentropy, metrics=['accuracy'])

# Train model

In [ ]:
model.fit(X_train, y_train_onehot, epochs=15, batch_size=32, verbose=1)

# Save the model

In [ ]:
model.save(model_save_path)

# Convert model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# Confusion matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

# Inference test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# Get tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Set tensors
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

#Inference Implementation

%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

